In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Any results you write to the current directory are saved as output.
from sklearn.multioutput import MultiOutputRegressor
import lightgbm as lgb
from tqdm import tqdm

In [12]:
ANTIBIOTIC_LIST = ['Amikacin', 'Ampicillin', 'Ampicillin/Sulbactam', 'Aztreonam', 'Cefazolin', 'Cefepime', 'Cefoxitin', 'Ceftazidime', 'Ceftriaxone', 'Cefuroxime sodium',
                   'Ciprofloxacin', 'Gentamicin', 'Imipenem', 'Levofloxacin', 'Meropenem', 'Nitrofurantoin', 'Piperacillin/Tazobactam', 'Tetracycline', 'Tobramycin', 'Trimethoprim/Sulfamethoxazole']


In [43]:
train = pd.read_csv("../data/datasets/2021-11-19 00:50:05.824288.csv", header=None)
test = pd.read_csv("../data/datasets/2021-11-24 00:24:56.435064.csv", header=None)

In [11]:
test_id = test.index.values
test_id

array([0, 1, 2, 3, 4])

In [94]:
reversed_list = ANTIBIOTIC_LIST[::-1]
y_list = []
index_names = []


for index, value in enumerate(reversed_list):
    index_name = int((len(train.columns) - 1) - index)
    
    y = train[index_name]

    if not len(y_list):
        y_list = np.zeros((len(y),len(reversed_list)))
    
    y_list[:,index] = y
    
    index_names.append(index_name)
    
print(y_list)



[[  4.   16.    4.  128.  128.    4.    8.    1.    2.    4.   32.   64.
   32.   32.   16.   32.   32.   32.   32.   16. ]
 [  4.   16.   16.    4.  128.    1.    8.    1.    2.    4.   32.   64.
   32.    4.   32.   32.   32.   32.   32.    8. ]
 [  2.   16.    4.  128.    0.   16.    8.   16.   16.    4.   32.   64.
   32.   32.    1.   32.   32.   32.   32.    8. ]
 [  4.    8.   16.  128.    0.   16.    8.   16.   16.    4.   32.   64.
   32.   32.   32.   32.   32.   32.   32.    8. ]
 [  4.    2.    4.   16.    0.    1.    8.    1.    2.    4.   32.   64.
   32.    8.    1.   32.   32.   32.   32.    8. ]
 [  4.    2.   16.   32.  128.    1.    8.    1.   16.    4.   32.   64.
   32.    4.   16.   32.   32.   32.   32.    8. ]
 [  0.5   2.    2.  128.    0.    1.    1.    1.    2.    1.   32.   64.
   32.    4.    8.   32.   32.   32.   32.    8. ]
 [  4.    8.   16.   64.    0.    1.    8.    1.    2.    4.   32.   64.
   32.    4.   32.   32.   32.   32.   32.    8. ]
 [  4.  

In [69]:
print(index_names)

[1383, 1382, 1381, 1380, 1379, 1378, 1377, 1376, 1375, 1374, 1373, 1372, 1371, 1370, 1369, 1368, 1367, 1366, 1365, 1364]


In [85]:
X_train = train.drop(index_names, axis=1)
X_test = test.drop(index_names, axis=1)

In [88]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,1354,1355,1356,1357,1358,1359,1360,1361,1362,1363
0,52771690,66853022,67096751,52667633,13532429,11857408,13010872,13870525,15572857,16993505,...,253835,234905,214428,296649,166621,189640,251871,373668,271961,454953
1,38683958,49628807,49828420,38645337,9727011,8704271,9581329,10310885,11510982,12530330,...,185886,164710,151179,226234,122772,132763,171470,262163,192123,304731
2,47582952,61723037,61896795,47558852,11913852,10824021,11817225,12577763,14287445,15690842,...,230203,201187,181736,278390,151319,161432,204142,327908,233605,375618
3,63315336,83655135,84139082,63079520,15750711,14472202,16080174,16555307,19169206,21418721,...,312355,269588,231648,356117,197673,211906,267585,434918,317684,513205
4,14350897,15861026,16175251,14228922,3982405,2910492,3252640,3980442,3890445,3883610,...,66793,70974,65481,77228,45569,59614,90049,108598,87007,156514
5,43329447,55271496,55473182,43251891,11029714,9776852,10712500,11401292,12806558,14036058,...,214390,193247,162733,248862,137987,146528,192293,308092,223806,362730
6,52455611,71629473,72073982,52285986,12805273,12419387,13208976,13506302,16048312,18568107,...,252634,214811,192290,304644,168532,170552,203638,339311,252353,421705
7,54213827,72026049,72571648,53977596,13682617,12210013,13747915,14168958,16346421,18301881,...,289057,235592,213266,337533,181649,186597,239449,388564,283477,449187
8,89154896,114113371,114692151,88911929,23011720,19822957,22150411,23688694,26299273,28935186,...,449931,415275,350119,516740,290380,325673,424871,659334,478402,798803
9,44504864,57446918,57649740,44467192,11236436,9986061,11001173,11865134,13243767,14477926,...,218101,188502,172648,262279,142071,152386,199930,308439,223868,362131


In [89]:
X_test

,0,1,2,3,4,5,6,7,8,9,...,1354,1355,1356,1357,1358,1359,1360,1361,1362,1363
0,33848343,43639935,43804265,33809132,8491258,7681343,8422020,8935014,10119167,11088846,...,162779,146615,129405,196505,106822,115593,145434,233835,166628,268784
1,48922071,61303046,61573469,48767343,12480550,11092586,12080936,12808447,14455028,15592262,...,235137,219813,181067,270502,150930,164696,218593,346484,251313,411368
2,46314529,57970294,58197713,46246605,11815630,10320294,11381017,12343268,13562019,14593875,...,224970,209118,174298,256487,156801,156481,216940,326100,233686,396580
3,29428430,37819137,37960413,29380221,7389067,6648686,7297257,7816443,8806340,9594421,...,139543,124830,112806,168524,92541,101214,128887,199549,144573,233049
4,45072036,56072104,56292169,45039341,11668432,10379241,10927007,11672560,13201627,14302760,...,202905,269760,242420,228985,182888,143026,238630,326819,254061,409722
5,42385610,56091511,56251628,42350942,10449650,9648550,10634658,11239101,12767079,14265669,...,207593,170629,159515,246154,133366,138943,175034,281896,202900,321114


In [80]:
y_list

array([[  4. ,  16. ,   4. , 128. , 128. ,   4. ,   8. ,   1. ,   2. ,
          4. ,  32. ,  64. ,  32. ,  32. ,  16. ,  32. ,  32. ,  32. ,
         32. ,  16. ],
       [  4. ,  16. ,  16. ,   4. , 128. ,   1. ,   8. ,   1. ,   2. ,
          4. ,  32. ,  64. ,  32. ,   4. ,  32. ,  32. ,  32. ,  32. ,
         32. ,   8. ],
       [  2. ,  16. ,   4. , 128. ,   0. ,  16. ,   8. ,  16. ,  16. ,
          4. ,  32. ,  64. ,  32. ,  32. ,   1. ,  32. ,  32. ,  32. ,
         32. ,   8. ],
       [  4. ,   8. ,  16. , 128. ,   0. ,  16. ,   8. ,  16. ,  16. ,
          4. ,  32. ,  64. ,  32. ,  32. ,  32. ,  32. ,  32. ,  32. ,
         32. ,   8. ],
       [  4. ,   2. ,   4. ,  16. ,   0. ,   1. ,   8. ,   1. ,   2. ,
          4. ,  32. ,  64. ,  32. ,   8. ,   1. ,  32. ,  32. ,  32. ,
         32. ,   8. ],
       [  4. ,   2. ,  16. ,  32. , 128. ,   1. ,   8. ,   1. ,  16. ,
          4. ,  32. ,  64. ,  32. ,   4. ,  16. ,  32. ,  32. ,  32. ,
         32. ,   8. ],
       [  

In [75]:
N = len(index_names)
N

20

In [82]:
y_pred = 0


In [109]:
for i in tqdm(range(N)):
    model = MultiOutputRegressor(lgb.LGBMRegressor(), n_jobs=-1)
    model.fit(X_train, y_list)
    y_pred += model.predict(X_train)

100%|██████████| 20/20 [00:01<00:00, 13.90it/s]


In [110]:
y_pred

array([[  496.,  1184.,  1760., 12352.,  8192.,   688.,  1056.,   640.,
          992.,   536.,  5120.,  9248.,  4736.,  2560.,  2736.,  5120.,
         4640.,  5120.,  5120.,  1408.],
       [  496.,  1184.,  1760., 12352.,  8192.,   688.,  1056.,   640.,
          992.,   536.,  5120.,  9248.,  4736.,  2560.,  2736.,  5120.,
         4640.,  5120.,  5120.,  1408.],
       [  496.,  1184.,  1760., 12352.,  8192.,   688.,  1056.,   640.,
          992.,   536.,  5120.,  9248.,  4736.,  2560.,  2736.,  5120.,
         4640.,  5120.,  5120.,  1408.],
       [  496.,  1184.,  1760., 12352.,  8192.,   688.,  1056.,   640.,
          992.,   536.,  5120.,  9248.,  4736.,  2560.,  2736.,  5120.,
         4640.,  5120.,  5120.,  1408.],
       [  496.,  1184.,  1760., 12352.,  8192.,   688.,  1056.,   640.,
          992.,   536.,  5120.,  9248.,  4736.,  2560.,  2736.,  5120.,
         4640.,  5120.,  5120.,  1408.],
       [  496.,  1184.,  1760., 12352.,  8192.,   688.,  1056.,   640.,
   